## Tranformar datase de IDS para utilizar apenas dimensão economica

In [223]:
import pandas as pd
from lxml import html
import requests
import zipfile
import io
import os
from pathlib import Path
import csv

In [224]:
df = pd.read_csv("http://ftp.dadosabertos.ans.gov.br/FTP/PDA/historico_idss/historico_idss.csv",sep=";",encoding="ISO-8859-1")

In [225]:
df.columns

Index(['Registro', 'CNPJ', 'Razao_Social', 'Modalidade', 'IDSS_2009_2008',
       'IDSS_2010_2009', 'IDSS_2011_2010', 'IDAS_2011_2010', 'IDEO_2011_2010',
       'IDEF_2011_2010', 'IDSB_2011_2010', 'IDSS_2012_2011', 'IDAS_2012_2011',
       'IDEO_2012_2011', 'IDEF_2012_2011', 'IDSB_2012_2011', 'IDSS_2013_2012',
       'IDAS_2013_2012', 'IDEO_2013_2012', 'IDEF_2013_2012', 'IDSB_2013_2012',
       'IDSS_2014_2013', 'IDAS_2014_2013', 'IDEO_2014_2013', 'IDEF_2014_2013',
       'IDSB_2014_2013', 'IDSS_2015_2014', 'IDAS_2015_2014', 'IDEO_2015_2014',
       'IDEF_2015_2014', 'IDSB_2015_2014', 'IDSS_2016_2015', 'IDQS_2016_2015',
       'IDGA_2016_2015', 'IDSM_2016_2015', 'IDGR_2016_2015', 'IDSS_2017_2016',
       'IDQS_2017_2016', 'IDGA_2017_2016', 'IDSM_2017_2016', 'IDGR_2017_2016',
       'IDSS_2018_2017', 'IDQS_2018_2017', 'IDGA_2018_2017', 'IDSM_2018_2017',
       'IDGR_2018_2017'],
      dtype='object')

In [226]:
df = df[['Registro','Razao_Social','Modalidade','IDEF_2012_2011','IDEF_2013_2012','IDEF_2014_2013','IDEF_2015_2014','IDSM_2016_2015','IDSM_2017_2016','IDSM_2018_2017']]

## Filtrando para obter apenas cooperativas

In [227]:
df_cooperativas = df[df['Modalidade'] == "Cooperativa Médica"]

In [228]:
df_cooperativas.dropna(inplace=True)

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [229]:
df_cooperativas.head()

,Registro,Razao_Social,Modalidade,IDEF_2012_2011,IDEF_2013_2012,IDEF_2014_2013,IDEF_2015_2014,IDSM_2016_2015,IDSM_2017_2016,IDSM_2018_2017
14,300136,UNIMED LITORAL SUL/RS - COOPERATIVA MÉDICA LTDA.,Cooperativa Médica,"0,8845","0,9519","0,6503","0,8333","0,7693","0,7882","0,6030"
18,300713,UNIMED DE ASSIS COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,"0,9245","0,9060","0,8900","1,0000","0,8875","0,9530","0,9640"
21,300870,UNIMED DO BRASIL - CONF. NACIONAL DAS COOPERAT...,Cooperativa Médica,ND,ND,ND,"1,0000",ND,"0,0000","0,5714"
25,301060,UNIMED ITUIUTABA COOPERATIVA TRABALHO MÉDICO L...,Cooperativa Médica,"0,8218","0,8914","0,9218","1,0000","0,7453","0,8240","0,9914"
26,301124,UNIMED GURUPI COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,"0,7956","0,9227","0,9069","1,0000","0,9890","0,5970","0,6660"


In [230]:
def converter_para_float(x):
    try:
        return float(x.replace(".","").replace(",","."))
    except:
        return 0
        

### Tranformando índices em variáveis do tipo float

In [231]:
colunas_indices = ['IDEF_2012_2011','IDEF_2013_2012','IDEF_2014_2013','IDEF_2015_2014','IDSM_2016_2015','IDSM_2017_2016','IDSM_2018_2017']
for coluna in colunas_indices:
    df_cooperativas[coluna] = df_cooperativas[coluna].apply(lambda x: converter_para_float(x))

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [232]:
df_cooperativas.head()

,Registro,Razao_Social,Modalidade,IDEF_2012_2011,IDEF_2013_2012,IDEF_2014_2013,IDEF_2015_2014,IDSM_2016_2015,IDSM_2017_2016,IDSM_2018_2017
14,300136,UNIMED LITORAL SUL/RS - COOPERATIVA MÉDICA LTDA.,Cooperativa Médica,0.8845,0.9519,0.6503,0.8333,0.7693,0.7882,0.6030
18,300713,UNIMED DE ASSIS COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,0.9245,0.9060,0.8900,1.0000,0.8875,0.9530,0.9640
21,300870,UNIMED DO BRASIL - CONF. NACIONAL DAS COOPERAT...,Cooperativa Médica,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.5714
25,301060,UNIMED ITUIUTABA COOPERATIVA TRABALHO MÉDICO L...,Cooperativa Médica,0.8218,0.8914,0.9218,1.0000,0.7453,0.8240,0.9914
26,301124,UNIMED GURUPI COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,0.7956,0.9227,0.9069,1.0000,0.9890,0.5970,0.6660


In [233]:
def converter_para_binario(x):
    for coluna in colunas_indices:
        valor_coluna = coluna.split("_")[-1]
#         print(coluna,valor_coluna)
        if (x[coluna] >= 0.8):
            x["utilizar_"+valor_coluna] = 1
        else:
            x["utilizar_"+valor_coluna] = 0
    return x

In [234]:
df_cooperativas= df_cooperativas.apply(lambda row: converter_para_binario(row) , axis=1)

In [235]:
df_cooperativas.drop(columns=colunas_indices,inplace=True)

In [236]:
df_cooperativas.columns

Index(['Registro', 'Razao_Social', 'Modalidade', 'utilizar_2011',
       'utilizar_2012', 'utilizar_2013', 'utilizar_2014', 'utilizar_2015',
       'utilizar_2016', 'utilizar_2017'],
      dtype='object')

In [237]:
df_cooperativas.head()

,Registro,Razao_Social,Modalidade,utilizar_2011,utilizar_2012,utilizar_2013,utilizar_2014,utilizar_2015,utilizar_2016,utilizar_2017
14,300136,UNIMED LITORAL SUL/RS - COOPERATIVA MÉDICA LTDA.,Cooperativa Médica,1,1,0,1,0,0,0
18,300713,UNIMED DE ASSIS COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,1,1,1,1,1,1,1
21,300870,UNIMED DO BRASIL - CONF. NACIONAL DAS COOPERAT...,Cooperativa Médica,0,0,0,1,0,0,0
25,301060,UNIMED ITUIUTABA COOPERATIVA TRABALHO MÉDICO L...,Cooperativa Médica,1,1,1,1,0,1,1
26,301124,UNIMED GURUPI COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,0,1,1,1,1,0,0


In [238]:
df_cooperativas.set_index("Registro",inplace=True)

In [239]:
df_porte = pd.read_csv("../entrada/porte_operadoras.csv",encoding="ISO-8859-1")
df_porte.drop(columns=['beneficiarios'],inplace=True)

In [240]:
df_porte["alto_nivel"] = 0

In [241]:
def marcar_registros_utilizacao(x):
    _ano = x['ano']
    valor = 0
    if (_ano in range(2011,2018)):
        nome_coluna = "utilizar_"+str(_ano)
        try:
            valor =  df_cooperativas.loc[x['Registro_ANS'],:][nome_coluna]
        except:
            pass
    return valor

In [242]:
df_porte['alto_nivel'] = df_porte.apply(lambda x: marcar_registros_utilizacao(x),axis=1)

In [243]:
if (not(os.path.exists('../entrada'))):
    os.mkdir('../entrada')

In [244]:
len(df_porte)

2547

In [245]:
df_porte = df_porte[df_porte['alto_nivel'] == 1]

In [250]:
len(df_porte)

1129

In [252]:
df_porte.to_csv("../entrada/idss_alto_nivel.csv",index=False,encoding="ISO-8859-1")

In [255]:
df_porte.groupby(['ano','porte'])['Registro_ANS'].count()

ano   porte        
2011  Grande Porte       5
      Médio Porte       26
      Pequeno Porte     68
2012  Grande Porte       8
      Médio Porte       49
      Pequeno Porte     70
2013  Grande Porte      10
      Médio Porte       44
      Pequeno Porte     75
2014  Grande Porte      30
      Médio Porte       92
      Pequeno Porte    106
2015  Grande Porte      16
      Médio Porte       62
      Pequeno Porte     90
2016  Grande Porte      14
      Médio Porte       67
      Pequeno Porte     98
2017  Grande Porte      22
      Médio Porte       72
      Pequeno Porte    105
Name: Registro_ANS, dtype: int64